# Fine-tuning a BERT model for text extraction with the SQuAD dataset

We are going to fine-tune [BERT implemented by HuggingFace](https://huggingface.co/bert-base-uncased) for the text-extraction task with a dataset of questions and answers with the [SQuAD (The Stanford Question Answering Dataset)](https://rajpurkar.github.io/SQuAD-explorer/) dataset.
The data is composed by a set of questions and corresponding paragraphs that contains the answers.
The model will be trained to locate the answer in the context by giving the positions where the answer starts and ends.

In this notebook we are going to do the training using multiple GPUs.

This notebook is based on [BERT (from HuggingFace Transformers) for Text Extraction](https://keras.io/examples/nlp/text_extraction_with_bert/).

More info:
- [Glossary - HuggingFace docs](https://huggingface.co/transformers/glossary.html#model-inputs)
- [BERT NLP — How To Build a Question Answering Bot](https://towardsdatascience.com/bert-nlp-how-to-build-a-question-answering-bot-98b1d1594d7b)

In [ ]:
import ipcmagic

In [ ]:
%ipcluster start -n 2

In [ ]:
%pxconfig --progress-after -1

In [ ]:
%%px
import os
import utility.data_processing as dpp
import utility.testing as testing
import torch
import torch.distributed as dist
from datasets import load_dataset, load_metric
from datetime import datetime
from transformers import BertTokenizer, BertForQuestionAnswering
from tokenizers import BertWordPieceTokenizer
from torch.utils.data import DataLoader, DistributedSampler
from torch.nn.parallel import DistributedDataParallel

In [ ]:
%%px
from datasets.utils import disable_progress_bar
from datasets import disable_caching


disable_progress_bar()
disable_caching()

In [ ]:
%%px
hf_model = 'bert-base-uncased'
bert_cache = os.path.join(os.getcwd(), 'cache')

In [ ]:
%%px
slow_tokenizer = BertTokenizer.from_pretrained(
    hf_model,
    cache_dir=os.path.join(bert_cache, f'_{hf_model}-tokenizer')
)
save_path = os.path.join(bert_cache, f'{hf_model}-tokenizer')
if not os.path.exists(save_path):
    os.makedirs(save_path)
    slow_tokenizer.save_pretrained(save_path)
    
# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer(os.path.join(save_path, 'vocab.txt'),
                                   lowercase=True)

In [ ]:
%%px
model = BertForQuestionAnswering.from_pretrained(
    hf_model,
    cache_dir=os.path.join(bert_cache, f'{hf_model}_qa')
)

In [ ]:
%%px
from pt_distr_env import setup_distr_env

setup_distr_env()
dist.init_process_group(backend="nccl")
rank = dist.get_rank()
world_size = dist.get_world_size()

In [ ]:
%%px
hf_dataset = load_dataset('squad')

In [ ]:
%%px
max_len = 384

In [ ]:
%%px
hf_dataset.flatten()
processed_dataset = hf_dataset.flatten().map(
    lambda example: dpp.process_squad_item_batched(example, max_len, tokenizer),
    remove_columns=hf_dataset.flatten()['train'].column_names,
    batched=True,
    num_proc=12
)

In [ ]:
%%px
train_dataset = processed_dataset["train"]
train_dataset.set_format(type='torch')

eval_dataset = processed_dataset["validation"]
eval_dataset.set_format(type='torch')

In [ ]:
%%px
per_device_train_batch_size = 16
per_device_eval_batch_size = 1

train_sampler = DistributedSampler(train_dataset, num_replicas=world_size,
                                   rank=rank, shuffle=False, seed=42)

train_dataloader = DataLoader(
    train_dataset,
    shuffle=False,  # sampler option is mutually exclusive with shuffle
    batch_size=per_device_train_batch_size,
    sampler=train_sampler
)

eval_dataloader = DataLoader(
    eval_dataset,
    shuffle=False,
    batch_size=per_device_eval_batch_size
)

In [ ]:
%%px
device = 0
model.to(device)
model = DistributedDataParallel(model, device_ids=[device])
model.train()

model.training

In [ ]:
%%px
optim = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [ ]:
%%px
for epoch in range(1):
    for i, batch in enumerate(train_dataloader):
        optim.zero_grad()
        outputs = model(input_ids=batch['input_ids'].to(device),
                        token_type_ids=batch['token_type_ids'].to(device),
                        attention_mask=batch['attention_mask'].to(device),
                        start_positions=batch['start_token_idx'].to(device),
                        end_positions=batch['end_token_idx'].to(device))        
        loss = outputs[0]
        loss.backward()
        optim.step()
        
        if i > 100:
            break

## Saving the model

In [ ]:
%%px --target 0
model_hash = datetime.now().strftime("%Y-%m-%d-%H%M%S")
model_path_name = f'./cache/model_trained_pytorch_{model_hash}'

# save model's state_dict
# the model now is a DDP model
# use `model.module.state_dict()` in order the load it later on
# any number of nodes
torch.save(model.module.state_dict(), model_path_name)

In [ ]:
%%px --target 0
# create the model again since the previous one is on the gpu
model_cpu = BertForQuestionAnswering.from_pretrained(
    "bert-base-uncased",
    cache_dir=os.path.join(bert_cache, 'bert-base-uncased_qa')
)

# load the model on cpu
model_cpu.load_state_dict(
    torch.load(model_path_name,
               map_location=torch.device('cpu'))
)

# load the model on gpu
# model.load_state_dict(torch.load(model_path_name))

In [ ]:
%%px --target 0
model.eval()

model.training

In [ ]:
%%px --target 0
squad_example_objects = []
for item in hf_dataset['validation'].flatten():
    squad_examples = dpp.squad_examples_from_dataset(item, max_len, tokenizer)
    try:
        squad_example_objects.extend(squad_examples)
    except TypeError:
        squad_example_objects.append(squad_examples)
        
assert len(eval_dataset) == len(squad_example_objects)

In [ ]:
%%px --target 0

start_sample = 0
num_test_samples = 10
for i, eval_batch in enumerate(eval_dataloader):
    if i > start_sample:
        testing.EvalUtility(eval_batch, [squad_example_objects[i]], model).results()

    if i > start_sample + num_test_samples:
        break

In [ ]:
%ipcluster stop